# Project Notebook: Augmenting Pandas with SQLite

## Question 1: Introduction

In this session, we explored a few different ways to work with larger datasets in pandas. In this guided project, we'll practice using some of the techniques we learned to analyze startup investments from Crunchbase.com.

Every year, thousands of startup companies raise financing from investors. Each time a startup raises money, we refer to the event as a fundraising round. Crunchbase is a website that crowdsources information on the fundraising rounds of many startups. The Crunchbase user community submits, edits, and maintains most of the information in Crunchbase.

In return, Crunchbase makes the data available through a Web application and a fee-based API. Before Crunchbase switched to the paid API model, multiple groups crawled the site and released the data online. Because the information on the startups and their fundraising rounds is always changing, the data set we'll be using isn't completely up to date.

Throughout this project, we'll practice working with different memory constraints. In this step, let's assume we only have 10 megabytes of available memory. While crunchbase-investments.csv (https://bit.ly/3BPcobU) consumes 10.3 megabytes of disk space, we know from earlier lessons that pandas often requires 4 to 6 times amount of space in memory as the file does on disk (especially when there's many string columns).


**Tasks**

* Because the data set contains over 50,000 rows, you'll need to read the data set into dataframes using 5,000 row chunks to ensure that each chunk consumes much less than 10 megabytes of memory.
* Across all of the chunks, become familiar with:
1. Each column's missing value counts.
2. Each column's memory footprint.
3. The total memory footprint of all of the chunks combined.
4. Which column(s) we can drop because they aren't useful for analysis.




In [5]:
# Dataset URL = https://bit.ly/3BPcob
import pandas as pd

import matplotlib.pyplot as plt

df = pd.read_csv("https://bit.ly/3BPcobU",encoding = 'unicode_escape')
df.shape


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(52870, 20)

In [6]:
# checking missing data per column in the entire network

df.isnull().sum()

company_permalink             1
company_name                  1
company_category_code       643
company_country_code          1
company_state_code          492
company_region                1
company_city                533
investor_permalink            2
investor_name                 2
investor_category_code    50427
investor_country_code     12001
investor_state_code       16809
investor_region               2
investor_city             12480
funding_round_type            3
funded_at                     3
funded_month                  3
funded_quarter                3
funded_year                   3
raised_amount_usd          3599
dtype: int64

In [7]:
# dividing data into chunks and checking null info in each chunk


chrunch_iter = pd.read_csv("https://bit.ly/3BPcobU",encoding = 'unicode_escape', chunksize=5000)
for chunk in chrunch_iter:

#calcualting memeory usage per chunk  
  print(chunk.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       5000 non-null   object 
 1   company_name            5000 non-null   object 
 2   company_category_code   4948 non-null   object 
 3   company_country_code    5000 non-null   object 
 4   company_state_code      4947 non-null   object 
 5   company_region          5000 non-null   object 
 6   company_city            4936 non-null   object 
 7   investor_permalink      5000 non-null   object 
 8   investor_name           5000 non-null   object 
 9   investor_category_code  2443 non-null   object 
 10  investor_country_code   4222 non-null   object 
 11  investor_state_code     3629 non-null   object 
 12  investor_region         5000 non-null   object 
 13  investor_city           4100 non-null   object 
 14  funding_round_type      5000 non-null   

## Observation:
investor_category_code have a lot of null values and can be dropped in this data .

the aim of the data is raised amount, and so, all null values of raised_amount_usd should be deleted

In [8]:
chrunch_iter = pd.read_csv("https://bit.ly/3BPcobU",encoding = 'unicode_escape', chunksize=5000)
for chunk in chrunch_iter:
    #drop rows where raised amount is null
  chunk = chunk.dropna(subset=['raised_amount_usd'])
  #drop colulmn investor_category_code
  chunk.drop(['investor_category_code'], axis=1)
  #calcualting memeory usage per chunk  
  print(chunk.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4347 entries, 0 to 4999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   company_permalink       4347 non-null   object 
 1   company_name            4347 non-null   object 
 2   company_category_code   4302 non-null   object 
 3   company_country_code    4347 non-null   object 
 4   company_state_code      4306 non-null   object 
 5   company_region          4347 non-null   object 
 6   company_city            4301 non-null   object 
 7   investor_permalink      4347 non-null   object 
 8   investor_name           4347 non-null   object 
 9   investor_category_code  1973 non-null   object 
 10  investor_country_code   3644 non-null   object 
 11  investor_state_code     3103 non-null   object 
 12  investor_region         4347 non-null   object 
 13  investor_city           3533 non-null   object 
 14  funding_round_type      4347 non-null   

In [9]:
# deleting the above unnecessary data
for chunk in chrunch_iter:
 
   #drop rows where raised amount is null
  chunk = chunk.dropna(subset=['raised_amount_usd'])
  #drop colulmn investor_category_code
  chunk.drop(['investor_category_code'], axis=1)
  #calcualting memeory usage per chunk  
  print(chunk.info())


## Question 2: Selecting Data Types

Now that we have a good sense of the missing values, let's get familiar with the column types before adding the data into SQLite.

**Tasks**

* Identify the types for each column.
* Identify the numeric columns we can represent using more space efficient types.
For text columns:
* Analyze the unique value counts across all of the chunks to see if we can convert them to a numeric type.
* See if we clean clean any text columns and separate them into multiple numeric columns without adding any overhead when querying.
* Make your changes to the code from the last step so that the overall memory the data consumes stays under 10 megabytes.


In [10]:
# converting objects with less than 0.5 unique values into category
for col in chunk.select_dtypes(include=['object']):
    num_unique_values = len(chunk[col].unique())
    num_total_values = len(chunk[col])
    if num_unique_values / num_total_values < 0.5:
        chunk[col] = chunk[col].astype('category')
        
print(chunk.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2580 entries, 50000 to 52869
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   company_permalink       2580 non-null   object  
 1   company_name            2580 non-null   object  
 2   company_category_code   2572 non-null   category
 3   company_country_code    2580 non-null   category
 4   company_state_code      2555 non-null   category
 5   company_region          2580 non-null   category
 6   company_city            2540 non-null   category
 7   investor_permalink      2580 non-null   category
 8   investor_name           2580 non-null   category
 9   investor_category_code  0 non-null      float64 
 10  investor_country_code   0 non-null      float64 
 11  investor_state_code     0 non-null      float64 
 12  investor_region         2580 non-null   category
 13  investor_city           0 non-null      float64 
 14  funding_round_type 

In [11]:
float_chunk = chunk.select_dtypes(include=['float64'])
print(float_chunk.isnull().sum())

investor_category_code    2580
investor_country_code     2580
investor_state_code       2580
investor_city             2580
raised_amount_usd            0
dtype: int64


## Question 3: Loading Chunks Into SQLite

Now we're in good shape to start exploring and analyzing the data. The next step is to load each chunk into a table in a SQLite database so we can query the full data set.

**Tasks**

1. Create and connect to a new SQLite database file.
2. Expand on the existing chunk processing code to export each chunk to a new table in the SQLite database.
3. Query the table and make sure the data types match up to what you had in mind for each column.

In [14]:
# Your code goes here

import sqlite3

#create new database chrunch 
conn = sqlite3.connect('chrunch.db')
for chunk in chrunch_iter:
#export the chunk data to the sqlite and call it lenders
  chunk.to_sql("lenders", conn, if_exists='append', index=False)

#querry table 

new_df = pd.read_sql('PRAGMA table_info(lenders);', conn)
print(new_df)


    cid                    name     type  notnull dflt_value  pk
0     0       company_permalink     TEXT        0       None   0
1     1            company_name     TEXT        0       None   0
2     2   company_category_code     TEXT        0       None   0
3     3    company_country_code     TEXT        0       None   0
4     4      company_state_code     TEXT        0       None   0
5     5          company_region     TEXT        0       None   0
6     6            company_city     TEXT        0       None   0
7     7      investor_permalink     TEXT        0       None   0
8     8           investor_name     TEXT        0       None   0
9     9  investor_category_code     REAL        0       None   0
10   10   investor_country_code     REAL        0       None   0
11   11     investor_state_code     REAL        0       None   0
12   12         investor_region     TEXT        0       None   0
13   13           investor_city     REAL        0       None   0
14   14      funding_roun

## Question 4: Next Steps

Now that the data is in SQLite, we can use the pandas SQLite workflow we learned in the last lesson to explore and analyze startup investments. Remember that each row isn't a unique company, but a unique investment from a single investor. This means that many startups will span multiple rows.

Use the pandas SQLite workflow to answer the following questions:

* What proportion of the total amount of funds did the top 10% raise? What about the top 1%? Compare these values to the proportions the bottom 10% and bottom 1% raised.
* Which category of company attracted the most investments?
* Which investor contributed the most money (across all startups)?
* Which investors contributed the most money per startup?
* Which funding round was the most popular? Which was the least popular?

Here are some ideas for further exploration:

* Repeat the tasks in this project using stricter memory constraints (under 1 megabyte).
* Clean and analyze the other Crunchbase data sets from the same GitHub repo.
* Understand which columns the data sets share, and how the data sets are linked.
* Create a relational database design that links the data sets together and reduces the overall disk space the database file consumes.

Use pandas to populate each table in the database, create the appropriate indexes, and so on.

In [32]:
# Your code goes here

#q = 'select exhibitionid, count(*) as counts from exhibitions group by exhibitionid order by counts desc;'
#eid_counts = pd.read_sql(q, conn)
#print(eid_counts[:10])

# Which category of company attracted the most investments?
c = 'select company_name,  count(*) as counts  from lenders group by  investor_name order by counts desc;'
company_name = pd.read_sql(c, conn)
print(company_name[:10])



  company_name  counts
0      Aisle50      94
1         Shyp      55
2    Remind101      48
3      LE TOTE      46
4      Swipely      43
5      Declara      37
6     Estimote      31
7      Uncovet      30
8      Koality      29
9       Vungle      28


Aisle50 attracted the most investments

In [36]:
#Which investor contributed the most money (across all startups)?
d = 'select investor_name, sum(raised_amount_usd) as sum from lenders group by investor_name order by sum desc;'
investment_amount = pd.read_sql(d, conn)
print(investment_amount[:10])

    investor_name          sum
0     Peter Thiel  538150000.0
1      Ron Conway  374544911.0
2  Scott Banister  231965000.0
3     Yuri Milner  207250000.0
4     Ted Leonsis  193050000.0
5    Reid Hoffman  191960000.0
6  Randal J. Kirk  150000000.0
7  Timothy Draper  144902000.0
8  Naval Ravikant  122075050.0
9      Steve Case  117102986.0


Peter Thiel contributed the most 

In [48]:
#Which investors contributed the most money per startup?
s = 'select investor_name,company_name, sum(raised_amount_usd) as sum from lenders group by company_name order by sum desc;'
startup_amount = pd.read_sql(s, conn)
print(startup_amount[:10])

              investor_name                     company_name          sum
0             Vikram Pandit                       CommonBond  300000000.0
1               Peter Thiel          Legendary Entertainment  275000000.0
2              Suhail Rizvi                        Flipboard  171000000.0
3            Randal J. Kirk             Intrexon Corporation  150000000.0
4               Ted Leonsis                 Revolution Money  134000000.0
5           Ross Perot, Jr.                          Clinkle  125000000.0
6  Stanley F. Druckenmiller             Relationship Science  120000000.0
7             Raymond James                  T5 Data Centers  113000000.0
8               Yuri Milner                          23andMe  112600000.0
9               Paul Posner  Pocket Communications Northeast  100000000.0


Vikram Pandit contrinuted most to a single commonBond startup company

In [46]:
#Which funding round was the most popular? Which was the least popular?
f = 'select funding_round_type,  count(*) as counts  from lenders group by  funding_round_type order by counts desc;'
funding_round = pd.read_sql(f, conn)
print(funding_round[:10])


  funding_round_type  counts
0              angel    1125
1           series-a     994
2            venture     192
3           series-b     145
4          series-c+      88
5              other      27
6     private-equity       9


most popular angel
last popular privae-equity